In [1]:
import sys
import numpy as np
import pandas as pd
import requests

def get_input(user_id):
    adress = 'http://35.180.178.243'

    req = requests.get(adress + '/reset?user_id=' + user_id)

    data = req.json()

    nb_users = data.pop('nb_users')
    nb_items = data.pop('nb_items')
    next_state = data.pop('next_state')
    columns = ['state_history', 'action_history', 'rewards_history']
    data_array = np.array([data.pop(c) for c in columns])
    df =  pd.DataFrame(data_array.T, columns=columns)
    return nb_users, nb_items, next_state, data_array

In [2]:
ID = 'VI2X71V0287S9F9B7SCU'

In [3]:
k = 10
t = []
for i in range(k):
    _, _, _ , data = get_input(ID)
    _, _, reward_history = data
    nb_reco = len(reward_history)
    good_reco = reward_history > 0
    nb_good_reco = np.sum(good_reco)
    t.append(100*nb_good_reco/nb_reco)

print('Distribution du taux de bonnes recommandations sur {} resets :'.format(k), t)
print('Taux moyen :', np.mean(t))
print('Ecart type :', np.std(t))

Distribution du taux de bonnes recommandations sur 10 resets : [30.5, 29.5, 31.0, 29.5, 32.0, 27.5, 25.0, 27.5, 29.0, 34.5]
Taux moyen : 29.6
Ecart type : 2.507987240796891


In [4]:
nb_users, nb_items, next_state , data = get_input(ID)

state_history, action_history, reward_history = data
columns_names = ['user_id', 'item_id', 'item_price'] + ['metadata_' + str(i) for i in range(5)]
dfs = [pd.DataFrame(state_history[i], columns=columns_names) for i in range(len(state_history))]
df = pd.DataFrame(state_history[0], columns=columns_names)
df['hist_line'] = 0
df['action_item_id'] = state_history[0][action_history[0]][1] 
df['reward'] = reward_history[0]
for i in range(1, len(state_history)):
    df_temp = pd.DataFrame(state_history[i], columns=columns_names)
    df_temp['hist_line'] = i
    df_temp['action_item_id'] = state_history[i][action_history[i]][1] 
    df_temp['reward'] = reward_history[i]
    df = df.append(df_temp, ignore_index=True)

df_next_state = pd.DataFrame(next_state, columns=columns_names)
user_df = pd.DataFrame(np.arange(nb_users), columns=['user_id']).set_index('user_id')
item_df = pd.DataFrame(np.arange(nb_items), columns=['item_id']).set_index('item_id')
df.head()

print('Nombre d\'utilisateurs :', nb_users)
print('Nombre d\'items :', nb_items)

nb_reco = len(reward_history)
good_reco = reward_history > 0
nb_good_reco = np.sum(good_reco)
nb_bad_reco = nb_reco - nb_good_reco
print('Nombre de recommandations :', nb_reco)
print('Nombre de bonnes recommandations :', nb_good_reco)
print('Nombre de mauvaises recommandations :', nb_bad_reco)
print('Taux de bonnes recommandations : {}%'.format(100*nb_bad_reco/nb_reco))

Nombre d'utilisateurs : 100
Nombre d'items : 30
Nombre de recommandations : 200
Nombre de bonnes recommandations : 65
Nombre de mauvaises recommandations : 135
Taux de bonnes recommandations : 67.5%


## Utilisateurs/items non présents dans l'historique

In [5]:
print('Nombre d\'utilisateurs non présents', nb_users-df['user_id'].nunique())
print('Nombre d\'items non présents', nb_items-df['item_id'].nunique())

Nombre d'utilisateurs non présents 12
Nombre d'items non présents 0


## Metadonnées utilisateur

In [6]:
for i in range(5):
    print('Metadata', i, ': ', (df.groupby('user_id')['metadata_' + str(i)].nunique() > 1).sum() == 0)

Metadata 0 :  True
Metadata 1 :  True
Metadata 2 :  False
Metadata 3 :  False
Metadata 4 :  False


Les metadonnées 0 et 1 sont des métadonnées utilisateurs (d'après l'historique)

## Metadonnées produits

In [7]:
for i in range(5):
    print('Metadata', i, ': ', (df.groupby('item_id')['metadata_' + str(i)].nunique() > 1).sum() == 0)

Metadata 0 :  False
Metadata 1 :  False
Metadata 2 :  True
Metadata 3 :  True
Metadata 4 :  False


Les metadonnées 2 et 3 sont des métadonnées utilisateurs (d'après l'historique)

## Nombre de commandes par utilisateurs

In [8]:
print('Nombre de commande par utilisateur minimum :', df.groupby('user_id')[['hist_line']].count().min()[0]) # 0 si on compte les absents
print('Nombre de commande par utilisateur maximum :', df.groupby('user_id')[['hist_line']].count().max()[0])

Nombre de commande par utilisateur minimum : 30
Nombre de commande par utilisateur maximum : 150


## Diversité des achats (quantitative)

In [9]:
bought_hist_idx = np.arange(len(state_history))[reward_history > 0]
df_bought = df[df.hist_line.isin(bought_hist_idx)]
df_bought = df_bought[df_bought.action_item_id == df_bought.item_id]
diversity = df_bought.groupby('user_id')[['item_id']].nunique()
number = df_bought.groupby('user_id')[['item_id']].nunique()

In [10]:
print('Nombre d\'articles par utilisateur minimum :', df.groupby('user_id')[['item_id']].nunique().min()[0]) # 0 si on compte les absents

Nombre d'articles par utilisateur minimum : 30


## Rapport au prix

In [11]:
#df[['user_id', 'item_price', 'metadata_2', 'metadata_3', 'metadata_4']].groupby('user_id').describe()
print('Minimum de la dépense moyenne :', df[['user_id', 'item_price']].groupby('user_id').mean().min()[0])
print('Maximum de la dépense moyenne :', df[['user_id', 'item_price']].groupby('user_id').mean().max()[0])
print('Minimum de l\'ecart type :', df[['user_id', 'item_price']].groupby('user_id').std().min()[0])
print('Maximum de l\'ecart type :', df[['user_id', 'item_price']].groupby('user_id').std().max()[0])


Minimum de la dépense moyenne : 473.30037747243875
Maximum de la dépense moyenne : 491.6626051383017
Minimum de l'ecart type : 277.96226475200683
Maximum de l'ecart type : 287.4167688690529


## Apparition de nouveaux utilisateurs ?

In [12]:
def send_pred(user_id, pred):
    adress = 'http://35.180.178.243'
    req = requests.get(adress + '/predict?user_id=' + user_id + '&recommended_item=' + str(pred))

    data = req.json()
    reward = data.pop('reward')
    state = data.pop('state')
    return reward, state

In [25]:
user_df = df.drop_duplicates(subset=['user_id']).sort_values('user_id')[['user_id', 'metadata_0', 'metadata_1']]

In [46]:
from numpy import linalg

norm = linalg.norm(user_df.values, axis = 1)
user_df['metadata_0_normed'] = user_df.metadata_0.values / norm
user_df['metadata_1_normed'] = user_df.metadata_1.values / norm

In [47]:
user_metadata = user_df[['metadata_0_normed', 'metadata_1_normed']].values

In [52]:
user_idx = 6
sim = user_metadata.dot(user_metadata[user_idx])
sim[user_idx] = -1

In [53]:
similar_to_user = np.argmax(sim)

In [54]:
user_df.user_id.iloc[similar_to_user]

1

In [55]:
sim

array([ 4.20278913e-02,  1.92176319e-01, -6.36937971e-02,  1.50494165e-02,
        1.56629768e-03,  4.99044681e-02, -1.00000000e+00,  2.84569842e-02,
        2.85611422e-02,  2.73056923e-02, -6.89752618e-03, -7.39784526e-03,
        8.18957435e-03,  6.16354262e-02,  4.75571829e-02,  3.71837723e-02,
        9.63460116e-03,  1.85047699e-02,  2.04169505e-02,  7.38093887e-03,
        3.63310749e-02,  1.52377964e-03,  2.98109823e-02,  1.37476555e-02,
        1.55754413e-02,  1.95750290e-03, -1.10432244e-02,  7.59236035e-03,
        1.01198732e-02,  1.35547370e-02,  1.06972400e-02,  1.37138863e-02,
        7.98465629e-03,  6.74169389e-03,  1.16441991e-02,  1.36779950e-02,
        7.68579508e-03, -1.65833166e-03,  5.39433125e-03,  1.32704673e-02,
        1.52236731e-02,  6.40025588e-03,  7.25460334e-03,  1.10003324e-02,
        4.75101279e-03,  1.18303937e-02,  1.02028072e-02,  1.00897799e-02,
        7.22388241e-04,  5.10963928e-03,  7.29202554e-03,  7.34656278e-03,
        1.62968193e-02,  